In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
from numpy import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from tensorflow import keras
import gc
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import f1_score,classification_report,roc_auc_score

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
plan_info = pd.read_csv('./ptom_third_plan.csv')
plan_info.dropna(how='all', inplace=True, axis=1)
plan_info.drop(['ad_start_time', 'ad_end_time', 'remark','origin'], axis=1, inplace=True)
plan_info.dropna(subset=['ad_info'],inplace=True)
plan_info.drop(['ad_id', 'ad_group_id', 'ad_name','media_id','plan_id','launch_op_id','bid_mode','bid_status','bid',
                'budget_mode','budget','opt_status','status','syn_time','update_time','inventory_type'], axis=1, inplace=True)
# 解析json
plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
temp = plan_info['ad_info'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('ad_info', axis=1, inplace=True)
temp = plan_info['audience'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('audience', axis=1, inplace=True)
temp = plan_info['action'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('action', axis=1, inplace=True)
plan_info.dropna(how='all', inplace=True, axis=1)

plan_info_2 = plan_info.copy()
plan_info_2.drop(['game_package_batch_id','advanced_creative_type','game_package_desc','game_package_thumbnail_ids',
               'external_url','intelligent_flow_switch','audience_package_id','union_video_type','converted_time_duration',
               'roi_goal','device_type','deep_bid_type','auto_extend_targets'],axis=1,inplace=True)
plan_info_2.drop(['package','open_url','ad_modify_time','deep_cpabid','download_url','download_mode','id','campaign_id',
               'package','ad_create_time','start_time','ad_id','name','status','modify_time','advertiser_id','schedule_type','quick_app_url',
               'end_time','convert_id','audit_reject_reason','download_type','budget_mode','bid','pricing',
               'include_custom_actions','app_type','exclude_flow_package','aweme_fan_behaviors','business_ids','user_type',
               'activate_type','aweme_fan_accounts','aweme_fan_categories','aweme_fans_numbers','article_category',
                'flow_package','carrier','superior_popularity_type','device_brand','exclude_custom_actions','geolocation',
               'opt_status','learning_phase','district'],axis=1,inplace=True)
# 删除老版兴趣行为定向
plan_info_2.drop(['ad_tag','interest_tags','app_behavior_target','app_category','app_ids'],axis=1,inplace=True)
# 删除老版兴趣行为定向
plan_info_2.drop(['retargeting_tags'],axis=1,inplace=True)

In [4]:
image_info = pd.read_csv('./image_info.csv')
# image_info.drop('create_time',axis=1,inplace=True)
# image_info.rename(columns={'create_time':'create_date'},inplace=True)

In [5]:
df = pd.merge(plan_info_2,image_info,on=['channel_id','source_id'],how='left')
df.dropna(subset=['image_id'],inplace=True)

In [6]:
launch_report = pd.read_csv('./launch_report.csv',usecols=['tdate','channel_id', 'source_id', 'amount',
       'create_role_num', 'pay_role_user_num', 'new_role_money'])
launch_report.sort_values(by='tdate',inplace=True)
launch_report.drop_duplicates(subset=['channel_id','source_id'],keep='first',inplace=True)
df = pd.merge(df, launch_report,on=['channel_id','source_id'],how='left')
df.drop(df[df['tdate'].isna()].index,inplace=True)

In [7]:
df['create_role_pay_cost'] = df.apply(lambda x:np.inf if x.pay_role_user_num==0 else x.amount/x.pay_role_user_num, axis=1)

In [8]:
df['platform'] = df['platform'].astype(str)
df['platform'] = df['platform'].map({"['ANDROID']":1,"['IOS']":2})

In [9]:
df['label'] = df.apply(lambda x: 1 if (x.pay_role_user_num>0)&(x.amount>=500) else 0,axis=1)

In [10]:
df['ad_account_id'] = df['ad_account_id'].astype('int')
df['image_id'] = df['image_id'].astype('int')
df.rename(columns={'tdate':'create_date'},inplace=True)
# df['create_date'] = pd.to_datetime(df['create_date']).dt.date
df['create_date'] = pd.to_datetime(df['create_date'])

In [11]:
# 读取image不同窗口期的运营数据
im_train_data_1 = pd.read_csv('./im_data/train_data_1.csv')
# im_train_data_2 = pd.read_csv('./im_data/train_data_2.csv')
im_train_data_3 = pd.read_csv('./im_data/train_data_3.csv')
# im_train_data_4 = pd.read_csv('./im_data/train_data_4.csv')
im_train_data_5 = pd.read_csv('./im_data/train_data_5.csv')
# im_train_data_6 = pd.read_csv('./im_data/train_data_6.csv')
im_train_data_7 = pd.read_csv('./im_data/train_data_7.csv')
# im_train_data_10 = pd.read_csv('./im_data/train_data_10.csv')
for i in [1,3,5,7]:
    exec("im_train_data_%d.drop(['label_ids_%d'],axis=1,inplace=True)"%(i,i))
    exec("im_train_data_%d.rename(columns={'model_run_datetime_%d':'create_date','image_id_%d':'image_id'},inplace=True)"%(i,i,i))
    exec("im_train_data_%d['image_id'] = im_train_data_%d['image_id'].astype('int')"%(i,i))
    exec("im_train_data_%d['create_date'] = pd.to_datetime(im_train_data_%d['create_date'])"%(i,i))

In [12]:
def fill_date(df,col):
    result_df = pd.DataFrame()
    for im_id in df[col].unique():
        temp = df[df[col]==im_id]
        im_date = temp['create_date'].values
        all_date = pd.date_range(temp['create_date'].min()- pd.DateOffset(days=1),temp['create_date'].max(),freq='D')
        new_date = pd.DataFrame(np.setdiff1d(all_date,im_date),columns=['create_date'])
        new_date[col]=im_id
        temp = pd.concat([temp,new_date])
        temp.sort_values(by='create_date',inplace=True)
        temp = temp.reset_index(drop=True)
        temp = temp.fillna(method='ffill')
        temp = temp.fillna(method='bfill')
        result_df = result_df.append(temp)
    return result_df

In [13]:
im_train_data_1 = fill_date(im_train_data_1,col='image_id')
im_train_data_3 = fill_date(im_train_data_3,col='image_id')
im_train_data_5 = fill_date(im_train_data_5,col='image_id')
im_train_data_7 = fill_date(im_train_data_7,col='image_id')

In [14]:
df = pd.merge(df, im_train_data_1, on=['image_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, im_train_data_2, on=['image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, im_train_data_3, on=['image_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, im_train_data_4, on=['image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, im_train_data_5, on=['image_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, im_train_data_6, on=['image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, im_train_data_7, on=['image_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, im_train_data_10, on=['image_id','create_date'],how='left', validate='many_to_one')

In [15]:
# 读取ad不同窗口期的运营数据
ad_train_data_1 = pd.read_csv('./ad_data/train_data_1.csv')
# ad_train_data_2 = pd.read_csv('./ad_data/train_data_2.csv')
ad_train_data_3 = pd.read_csv('./ad_data/train_data_3.csv')
# ad_train_data_4 = pd.read_csv('./ad_data/train_data_4.csv')
ad_train_data_5 = pd.read_csv('./ad_data/train_data_5.csv')
# ad_train_data_6 = pd.read_csv('./ad_data/train_data_6.csv')
ad_train_data_7 = pd.read_csv('./ad_data/train_data_7.csv')
# ad_train_data_10 = pd.read_csv('./ad_data/train_data_10.csv')
for i in [1,3,5,7]:
    exec("ad_train_data_%d.rename(columns={'model_run_datetime':'create_date'},inplace=True)"%(i))
    exec("ad_train_data_%d['ad_account_id'] = ad_train_data_%d['ad_account_id'].astype('int')"%(i,i))
    exec("ad_train_data_%d['create_date'] = pd.to_datetime(ad_train_data_%d['create_date'])"%(i,i))

In [16]:
ad_train_data_1 = fill_date(ad_train_data_1,col='ad_account_id')
ad_train_data_3 = fill_date(ad_train_data_3,col='ad_account_id')
ad_train_data_5 = fill_date(ad_train_data_5,col='ad_account_id')
ad_train_data_7 = fill_date(ad_train_data_7,col='ad_account_id')

In [17]:
df = pd.merge(df, ad_train_data_1, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, ad_train_data_2, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_train_data_3, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, ad_train_data_4, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_train_data_5, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, ad_train_data_6, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_train_data_7, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
# df = pd.merge(df, ad_train_data_10, on=['ad_account_id','create_date'],how='left', validate='many_to_one')

In [ ]:
# 读取ad_im不同窗口期的运营数据
ad_im_train_data_1 = pd.read_csv('./ad_im_data/train_data_1.csv')
ad_im_train_data_2 = pd.read_csv('./ad_im_data/train_data_2.csv')
ad_im_train_data_3 = pd.read_csv('./ad_im_data/train_data_3.csv')
ad_im_train_data_4 = pd.read_csv('./ad_im_data/train_data_4.csv')
ad_im_train_data_5 = pd.read_csv('./ad_im_data/train_data_5.csv')
ad_im_train_data_6 = pd.read_csv('./ad_im_data/train_data_6.csv')
ad_im_train_data_7 = pd.read_csv('./ad_im_data/train_data_7.csv')

In [ ]:
for i in [1,2,3,4,5,6,7]: 
    exec("ad_im_train_data_%d.rename(columns={'model_run_datetime':'create_date','image_id_%d':'image_id'},inplace=True)"%(i,i))
    exec("ad_im_train_data_%d['image_id'] = ad_im_train_data_%d['image_id'].astype('int')"%(i,i))
    exec("ad_im_train_data_%d['ad_account_id'] = ad_im_train_data_%d['ad_account_id'].astype('int')"%(i,i))
    exec("ad_im_train_data_%d['create_date'] = pd.to_datetime(ad_im_train_data_%d['create_date'])"%(i,i))

In [ ]:
df = pd.merge(df, ad_im_train_data_1, on=['ad_account_id','image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_im_train_data_2, on=['ad_account_id','image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_im_train_data_3, on=['ad_account_id','image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_im_train_data_4, on=['ad_account_id','image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_im_train_data_5, on=['ad_account_id','image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_im_train_data_6, on=['ad_account_id','image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, ad_im_train_data_7, on=['ad_account_id','image_id','create_date'],how='left', validate='many_to_one')

In [ ]:
cost_col = ['im_create_role_cost_1','im_create_role_cost_3',
            'im_create_role_cost_5','im_create_role_cost_7',
            'ad_create_role_cost_1','ad_create_role_cost_3',
            'ad_create_role_cost_5','ad_create_role_cost_7']
for col in cost_col:
    df[col] = df[col].apply(lambda x:x if x>0 else np.inf)

In [ ]:
df['create_time'] = pd.to_datetime(df['create_time'])
df['ad_im_sort_id'] = df.groupby(['ad_account_id','image_id'])['create_time'].rank()
df['ad_game_sort_id'] = df.groupby(['ad_account_id','game_id'])['create_time'].rank()
df['im_ad_sort_id'] = df.groupby(['image_id','ad_account_id'])['create_time'].rank()
df['weekday'] = df['create_time'].dt.weekday
df['month'] = df['create_time'].dt.month

In [ ]:
holidays = ['2020-01-01','2020-01-24','2020-01-25','2020-01-26','2020-01-27','2020-01-28',
           '2020-01-29','2020-01-30','2020-01-31','2020-04-04','2020-04-05','2020-04-06',
           '2020-05-01','2020-05-02','2020-05-03','2020-05-04','2020-05-05','2020-06-25',
           '2020-06-26','2020-06-27','2020-10-01','2020-10-02','2020-10-03','2020-10-04','2020-10-05',
           '2020-10-06','2020-10-07','2020-10-08','2021-01-01','2021-01-02','2021-01-03','2021-02-11',
           '2021-02-12','2021-02-13','2021-02-14','2021-02-15','2021-02-16','2021-02-17']

In [ ]:
df['create_date'] = pd.to_datetime(df['create_time']).dt.date
df['create_date'] = df['create_date'].astype(str)
df['is_holiday'] = (df.create_date.isin(holidays)).astype(int)

In [ ]:
df.drop(['interest_words','action_words','budget','bid2','cpa_bid'],axis=1,inplace=True)

df.drop(['create_role_num','amount','pay_role_user_num','new_role_money','create_role_pay_cost'],axis=1,inplace=True)
df.drop(['channel_id','source_id','create_date'],axis=1,inplace=True)

In [ ]:
# 对列表内容进行编码降维
def get_mutil_feature(data):
    cols = ['inventory_type','age','city','retargeting_tags_include','retargeting_tags_exclude','ac','interest_categories',
       'action_scene','action_categories']
    for col in cols:
        if col in ['inventory_type','age']:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
        elif col in ['city','retargeting_tags_include','retargeting_tags_exclude','interest_categories','action_categories']:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data[col]=data[col].apply(lambda x:[str(i) for i in x])
            temp = data[col].str.join('|').str.get_dummies()
    #         print(temp.shape[1])
            pca = PCA(n_components=0.9)
            temp = pca.fit_transform(temp.values)
            temp = pd.DataFrame(temp,columns = [col+str(i) for i in range(temp.shape[1])])
    #         print(temp.shape[1])
            data = data.join(temp)
            del temp
            data.drop(col,axis=1,inplace=True)
        else:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data[col]=data[col].apply(lambda x:[str(i) for i in x])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
    
    gc.collect()
    return data

In [ ]:
%%time
df = get_mutil_feature(df)

In [ ]:
cat_cols = ['ad_account_id','game_id','schedule_time','delivery_range','flow_control_mode',
            'smart_bid_type','hide_if_converted','gender','location_type','launch_price','retargeting_type',
            'android_osv','ios_osv','interest_action_mode',
            'action_days','image_id','label_ids']
for col in cat_cols:
    df[col] = df[col].astype(str)
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
# cat_cols = ['ad_account_id','game_id','inventory_type','schedule_time','delivery_range','flow_control_mode',
#             'smart_bid_type','hide_if_converted','age','gender','city','location_type','platform','launch_price','retargeting_type',
#             'retargeting_tags_include','retargeting_tags_exclude','ac','android_osv','ios_osv','interest_action_mode',
#             'interest_categories','action_categories','action_days','action_scene','image_id','label_ids']
# for col in cat_cols:
#     df[col] = df[col].astype(str)
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col])

In [ ]:
test_data = df[df['create_time'] >='2020-11-15']
train_data = df[df['create_time'] <'2020-11-15']

In [ ]:
test_data.drop(['create_time'],axis=1,inplace=True)
train_data.drop(['create_time'],axis=1,inplace=True)

In [ ]:
# train_data, test_data = train_test_split(df, test_size=0.25)
target = train_data['label']
features = train_data.drop(['label'], axis=1)
X_val, x_test, Y_val, y_test = train_test_split(features,target, test_size=0.3)

In [ ]:
# 不调参
params = {
"objective": "binary",
"boosting_type": "gbdt",
"learning_rate": 0.01,
"max_depth":8,
"num_leaves":55,
"max_bin":255,
"min_data_in_leaf":101,
"min_child_samples":15,
"feature_fraction": 0.5,
"bagging_fraction":0.6,
"bagging_freq":20,
"lambda_l1":1e-05,
"lambda_l2":0,
"min_split_gain": 0.0,
"metric": "auc",
'is_unbalance':True    
}

train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_b = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

In [ ]:
# 用train_data_pay_7_test数据集进行测试
target_test = test_data['label']
features_test = test_data.drop(['label'], axis=1)
y_predict = lgb_b.predict(features_test)

In [ ]:
sns.distplot(y_predict)
plt.show()

In [ ]:
roc_auc_score(target_test,y_predict)

In [ ]:
s = pd.Series(y_predict)
s.sort_values(ascending=False).reset_index(drop=True)[int(y_predict.shape[0] *0.07)]

In [ ]:
y_predict = list(map(lambda x:1 if x>=s.sort_values(ascending=False).reset_index(drop=True)[int(y_predict.shape[0] *0.07)] else 0,y_predict))

In [ ]:
print(classification_report(target_test, y_predict))

In [ ]:
plt.figure(figsize=(12,6))
lgb.plot_importance(lgb_b, max_num_features=20)
plt.title('Feature Importance')
plt.show()